Packages importeren

In [35]:
import pandas as pd
import numpy as np
import datetime as dt

ModuleNotFoundError: No module named 'pandas'

Brongegevens inlezen

In [13]:
DB = {'servername': 'ENVY-AKRAM\SQLEXPRESS',
'database': 'Great Outdoors - Akram en Sylvan'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\akram\AppData\Local\Temp\ipykernel_77496\1601167383.py:1: SyntaxWarning: invalid escape sequence '\S'
  DB = {'servername': 'ENVY-AKRAM\SQLEXPRESS',
C:\Users\akram\AppData\Local\Temp\ipykernel_77496\1601167383.py:1: SyntaxWarning: invalid escape sequence '\S'
  DB = {'servername': 'ENVY-AKRAM\SQLEXPRESS',


NameError: name 'pyodbc' is not defined